In [1]:
import json
import gc
import random
import networkx as nx
from networkx.readwrite import json_graph
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
import scml
import mylib

In [2]:
# number of context words in the left and right directions
m = 16
iters = 1
limit = 0
max_choices = 1000
type2id = {
    "click": 0,
    "cart": 1,
    "order": 2,
}

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [4]:
%%time
with open("input/vocab3.json") as f:
    id2label = [w for w, c in json.load(f)]
label2id = {k:v for v, k in enumerate(id2label)}
print(f"len(label2id)={len(label2id):,}")

len(label2id)=1,855,603
Wall time: 2.35 s


In [5]:
del id2label
gc.collect()

21

In [6]:
%%time
with open("input/graph.json") as f:
    g = nx.adjacency_graph(json.load(f))
print(g)

DiGraph with 3764159 nodes and 100953434 edges
Wall time: 9min 13s


In [7]:
rows = []
_nodes = g.nodes
if limit>0:
    _nodes = list(_nodes)[:limit]
out_edges = {}
in_edges = {}
for i in range(iters):
    for center in tqdm(_nodes, desc=f"i={i}"):
        # going left: predecessor nodes
        center_word, center_type = tuple(center.split("_"))
        curr = center
        for _ in range(m):
            if curr not in in_edges:
                choices = []
                for u, v, d in g.in_edges(curr, data=True):
                    if len(choices)>max_choices:
                        break
                    choices.append((u, d["weight"]))
                in_edges[curr] = choices
            choices = in_edges[curr]
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            if curr==center:
                break
            curr_word, curr_type = tuple(curr.split("_"))
            if curr_word!=center_word:
                rows.append({
                    "center_word": label2id[center_word],
                    "center_type": type2id[center_type],
                    "outside_word": label2id[curr_word],
                    "outside_type": type2id[curr_type],
                })
        # going right: successor nodes
        curr = center
        for _ in range(m):
            if curr not in out_edges:
                choices = []
                for u, v, d in g.out_edges(curr, data=True):
                    if len(choices)>max_choices:
                        break
                    choices.append((v, d["weight"]))
                out_edges[curr] = choices
            choices = out_edges[curr]
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            if curr==center:
                break
            curr_word, curr_type = tuple(curr.split("_"))
            if curr_word!=center_word:
                rows.append({
                    "center_word": label2id[center_word],
                    "center_type": type2id[center_type],
                    "outside_word": label2id[curr_word],
                    "outside_type": type2id[curr_type],
                })

i=0: 100%|██████████████████| 3764159/3764159 [5:29:00<00:00, 190.68it/s]


In [8]:
df = pd.DataFrame.from_records(rows)
df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
cols = ["center_word", "outside_word"]
df[cols] = df[cols].astype(np.int32)
cols = ["center_type", "outside_type"]
df[cols] = df[cols].astype(np.int8)
df.info()
assert (df["center_word"]!=df["outside_word"]).all()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109150008 entries, 0 to 109150007
Data columns (total 4 columns):
 #   Column        Dtype
---  ------        -----
 0   center_word   int32
 1   center_type   int8 
 2   outside_word  int32
 3   outside_type  int8 
dtypes: int32(2), int8(2)
memory usage: 1.0 GB


In [9]:
df.describe(percentiles=percentiles)

,center_word,center_type,outside_word,outside_type
count,1.091500e+08,1.091500e+08,1.091500e+08,1.091500e+08
mean,7.882119e+05,6.716308e-01,2.044651e+05,1.639858e-01
std,5.174330e+05,7.519884e-01,3.455319e+05,4.588682e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1%,1.295800e+04,0.000000e+00,2.000000e+01,0.000000e+00
5%,6.452800e+04,0.000000e+00,2.060000e+02,0.000000e+00
10%,1.300770e+05,0.000000e+00,6.790000e+02,0.000000e+00
20%,2.656770e+05,0.000000e+00,3.176000e+03,0.000000e+00
30%,4.093580e+05,0.000000e+00,9.437000e+03,0.000000e+00
40%,5.629080e+05,0.000000e+00,2.222000e+04,0.000000e+00


In [10]:
df.head(30)

,center_word,center_type,outside_word,outside_type
0,1,1,103,0
1,1,1,954,0
2,1,1,19250,0
3,1,1,43687,0
4,1,1,10334,0
5,1,1,55464,0
6,1,1,2173,0
7,1,1,9017,0
8,1,1,1389940,0
9,1,1,714759,0


In [11]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/pairs.parquet", index=False)

Wall time: 5.98 s


In [12]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 5:41:51.918474
